In [34]:
import numpy as np
import sklearn as skl
import pandas as pd
import sklearn.preprocessing
from sklearn import linear_model
from scipy import special
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import keras
from keras.layers import Dense, BatchNormalization
from keras.models import Sequential
from tensorflow.python.client import device_lib

np.random.seed(69) #fixing seed for reproducability

In [35]:
#print(device_lib.list_local_devices())

df_train = pd.read_csv("train.csv")
df_splitt = df_train['Sequence'].apply(lambda x: pd.Series(list(x))) #splitts the 4 mutators into individual columns.

#df_letters = pd.concat([df_train['Sequence'].apply(lambda x: pd.Series(list(x))), df_train['Active']],axis=1)

In [36]:
df_test = pd.read_csv("test.csv")
df_test_s = df_test['Sequence'].apply(lambda x: pd.Series(list(x)))

In [37]:
# define universe of possible input values
mutators = 'ACDEFGHIKLMNPQRSTUVWY' #abcdefghijklmnopqrstuvwxyz
#bjoqxz <-- these letters are missing

# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(mutators))
int_to_char = dict((i, c) for i, c in enumerate(mutators))

# integer encode input data
#integer_encoded = [char_to_int[item] for item in df_letters[0]]
#print(integer_encoded)

In [38]:
df = np.zeros((112000, 4))
#Possibly the worst fucking way we could have done this. But fuckem.
for i in range(0,4):
    for j in range(0, 112000):
        temp = char_to_int[df_splitt.loc[j,i]]
        #df[j, i*19 + temp] = 1
        df[j, i] = temp

In [39]:
#Do the same with the test dataframe
df_cock = np.zeros((48000, 4))

for i in range(0,4):
    for j in range(0, 48000):
        temp = char_to_int[df_test_s.loc[j,i]]
        #df_cock[j, i*19 + temp] = 1
        df_cock[j, i] = temp

In [40]:
clf = skl.linear_model.RidgeCV(alphas=[0.01, 0.01, 0.1, 1, 10, 100, 1000, 2000,5000], cv=15)

labels = df_train['Active'].to_numpy()

#predict cocks:
#clf.fit(df,labels)
print('Done with fitting')
#predict_train = clf.predict(df)
#print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
#predict_cancer = clf.predict(df_cock)
#predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

Done with fitting


In [41]:
#clf = skl.svm.LinearSVC(dual=False, class_weight='balanced')

#labels = df_train['Active'].to_numpy()

#predict cocks:
#clf.fit(df,labels)
print('Done with fitting')
#predict_train = clf.predict(df)
#print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
#predict_cancer = clf.predict(df_cock)
#predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

Done with fitting


In [42]:
#np.savetxt('submission_sklearn.csv', predict_cancer, delimiter=',', fmt='%i')

In [50]:
#lets norm this shit
df_nn = (df+1)/21
df_cock_nn = (df_cock+1)/21

print(df_nn.max(), df_nn.min())
print(labels.max(), labels.min())
print(df_cock_nn.max(), df_cock_nn.min())

1.0 0.047619047619047616
1 0
1.0 0.047619047619047616


In [44]:
#lets construct the neural net:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(100, activation='relu', input_dim=4))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(100, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(100, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(1, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(df_nn, labels, epochs=100, batch_size=256, verbose=1)

Epoch 1/100
438/438 [==============================] - 1s 3ms/step - loss: 0.6738 - accuracy: 0.9387

In [54]:
predict_cancer_nn = neuralNetwork.predict(df_cock_nn)
print(predict_cancer_nn)
predict_cancer_nn = (predict_cancer_nn <= -8)

predict_cancer_nn = pd.DataFrame(predict_cancer_nn.astype('int'))
predict_cancer_nn.to_csv('predictions.csv', header = False, index = False)

[[ -3.786911 ]
 [ -5.317009 ]
 [ -5.2450027]
 ...
 [ -4.6423035]
 [ -4.4252777]
 [-39.016785 ]]


In [48]:
print(predict_cancer_nn.max())
print(predict_cancer_nn.mean())

predict_train = neuralNetwork.predict(df_nn)
predict_train_nn = pd.DataFrame(predict_train.astype('float'))
print(predict_train_nn.max())
print(predict_train_nn.min())

0    1
dtype: int32
0    0.000125
dtype: float64
0    73.095268
dtype: float64
0   -61.916786
dtype: float64
